<a href="https://colab.research.google.com/github/leniartemily/QB1/blob/main/NFL_EL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

ModuleNotFoundError: ignored

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Starter.csv to Starter (2).csv


In [ ]:
import pandas as pd
import io
  
df = pd.read_csv('Starter.csv', header= 0,
                        encoding= 'unicode_escape')
df.head()

,Year,Name,College,POS,ProDay/Combine,Height (in),Weight (lbs),Wonderlic,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone,Started
0,2021,Ian Book,Notre Dame,QB,ProDay,72.00,211,NaN,4.65,NaN,32.5,115.0,4.20,7.00,No
1,2021,Shane Buechele,Southern Methodist (TX),QB,ProDay,72.25,210,NaN,4.96,NaN,30.0,114.0,4.60,7.34,No
2,2021,K.J. Costello,Mississippi State,QB,ProDay,76.50,225,NaN,5.11,NaN,27.5,103.0,4.75,NaN,No
3,2021,Sam Ehlinger,Texas,QB,ProDay,73.13,220,NaN,4.84,NaN,36.5,114.0,4.44,7.15,No
4,2021,Justin Fields,Ohio State,QB,ProDay,74.75,227,NaN,4.46,NaN,NaN,119.0,NaN,NaN,Yes


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727 entries, 0 to 726
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             727 non-null    int64  
 1   Name             727 non-null    object 
 2   College          727 non-null    object 
 3   POS              727 non-null    object 
 4   ProDay/Combine   727 non-null    object 
 5   Height (in)      727 non-null    float64
 6   Weight (lbs)     727 non-null    int64  
 7   Wonderlic        218 non-null    float64
 8   40 Yard          655 non-null    float64
 9   Bench Press      46 non-null     float64
 10  Vert Leap (in)   620 non-null    float64
 11  Broad Jump (in)  623 non-null    float64
 12  Shuttle          596 non-null    float64
 13  3Cone            438 non-null    float64
 14  Started          727 non-null    object 
dtypes: float64(8), int64(2), object(5)
memory usage: 85.3+ KB


In [ ]:
df.drop(columns=["Wonderlic","Bench Press","3Cone"],axis=1, inplace=True)

In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 573 entries, 0 to 726
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             573 non-null    int64  
 1   Name             573 non-null    object 
 2   College          573 non-null    object 
 3   POS              573 non-null    object 
 4   ProDay/Combine   573 non-null    object 
 5   Height (in)      573 non-null    float64
 6   Weight (lbs)     573 non-null    int64  
 7   40 Yard          573 non-null    float64
 8   Vert Leap (in)   573 non-null    float64
 9   Broad Jump (in)  573 non-null    float64
 10  Shuttle          573 non-null    float64
 11  Started          573 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 58.2+ KB


In [ ]:
df.drop(columns=["Name","Year","POS"],axis=1, inplace=True)

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 573 entries, 0 to 726
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   College          573 non-null    object 
 1   ProDay/Combine   573 non-null    object 
 2   Height (in)      573 non-null    float64
 3   Weight (lbs)     573 non-null    int64  
 4   40 Yard          573 non-null    float64
 5   Vert Leap (in)   573 non-null    float64
 6   Broad Jump (in)  573 non-null    float64
 7   Shuttle          573 non-null    float64
 8   Started          573 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 44.8+ KB


In [ ]:
X = pd.get_dummies(df)
X

,Height (in),Weight (lbs),40 Yard,Vert Leap (in),Broad Jump (in),Shuttle,College_Akron,College_Alabama,College_Alabama State,College_Alabama-Birmingham,College_Appalachian State (NC),College_Arizona,College_Arizona State,College_Arkansas,College_Arkansas State,College_Auburn,College_Ball State (IN),College_Baylor,College_Boise State,College_Boston College,College_Bowling Green (OH),College_Brigham Young,College_Buffalo,College_California,College_California (PA),College_Catholic University (DC),College_Central Arkansas,College_Central Connecticut,College_Central Florida,College_Central Michigan,College_Central Missouri,College_Central State (OH),College_Central Washington,College_Cincinnati,College_Clemson,College_Coastal Carolina (SC),College_Colorado,College_Colorado State,College_Colorado State-Pueblo,College_Connecticut,...,College_Texas,College_Texas A&M,College_Texas Christian,College_Texas State,College_Texas Tech,College_Texas-El Paso,College_Tiffin (OH),College_Toledo,College_Troy (AL),College_Tulane (LA),College_Tulsa (OK),College_UC-Davis,College_UCLA,College_Utah,College_Utah State,College_Vanderbilt,College_Virginia,College_Virginia Tech,College_Wake Forest,College_Washington,College_Washington State,College_Weber State (UT),College_West Texas A&M,College_West Virginia,College_Western Carolina,College_Western Kentucky,College_Western Michigan,College_Western New Mexico,College_Western State Colorado,College_William & Mary (VA),College_Williams College (MA),College_Wisconsin,College_Wisconsin-Steven's Point,College_Wyoming,College_Yale,ProDay/Combine_Both,ProDay/Combine_Combine,ProDay/Combine_ProDay,Started_No,Started_Yes
0,72.00,211,4.65,32.5,115.0,4.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,72.25,210,4.96,30.0,114.0,4.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,76.50,225,5.11,27.5,103.0,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,73.13,220,4.84,36.5,114.0,4.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
5,78.63,234,4.61,32.5,117.0,4.22,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,73.50,204,4.96,24.0,99.0,4.22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
721,74.00,201,5.13,26.5,99.0,4.49,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
722,75.60,207,5.15,26.5,104.0,4.63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
723,75.00,212,4.95,28.0,108.0,4.51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [ ]:
X.drop(columns=["Started_No"],axis=1, inplace=True)

In [ ]:
X.head()

,Height (in),Weight (lbs),40 Yard,Vert Leap (in),Broad Jump (in),Shuttle,College_Akron,College_Alabama,College_Alabama State,College_Alabama-Birmingham,College_Appalachian State (NC),College_Arizona,College_Arizona State,College_Arkansas,College_Arkansas State,College_Auburn,College_Ball State (IN),College_Baylor,College_Boise State,College_Boston College,College_Bowling Green (OH),College_Brigham Young,College_Buffalo,College_California,College_California (PA),College_Catholic University (DC),College_Central Arkansas,College_Central Connecticut,College_Central Florida,College_Central Michigan,College_Central Missouri,College_Central State (OH),College_Central Washington,College_Cincinnati,College_Clemson,College_Coastal Carolina (SC),College_Colorado,College_Colorado State,College_Colorado State-Pueblo,College_Connecticut,...,College_Tennessee-Chattanooga,College_Texas,College_Texas A&M,College_Texas Christian,College_Texas State,College_Texas Tech,College_Texas-El Paso,College_Tiffin (OH),College_Toledo,College_Troy (AL),College_Tulane (LA),College_Tulsa (OK),College_UC-Davis,College_UCLA,College_Utah,College_Utah State,College_Vanderbilt,College_Virginia,College_Virginia Tech,College_Wake Forest,College_Washington,College_Washington State,College_Weber State (UT),College_West Texas A&M,College_West Virginia,College_Western Carolina,College_Western Kentucky,College_Western Michigan,College_Western New Mexico,College_Western State Colorado,College_William & Mary (VA),College_Williams College (MA),College_Wisconsin,College_Wisconsin-Steven's Point,College_Wyoming,College_Yale,ProDay/Combine_Both,ProDay/Combine_Combine,ProDay/Combine_ProDay,Started_Yes
0,72.00,211,4.65,32.5,115.0,4.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,72.25,210,4.96,30.0,114.0,4.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,76.50,225,5.11,27.5,103.0,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,73.13,220,4.84,36.5,114.0,4.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,78.63,234,4.61,32.5,117.0,4.22,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
y=X["Started_Yes"]
X.drop(columns=["Started_Yes"],axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
classifier.fit(X_train, y_train)
print(model)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7855477855477856
Testing Data Score: 0.6805555555555556


In [ ]:
def getTestData(Height, Weight, FortyYard, VertLeap, BroadJump,	Shuttle, College):
  testData = [Height, Weight, FortyYard, VertLeap, BroadJump,	Shuttle, 0, 1, 0, 0]
  return testData

In [ ]:
predictions = classifier.predict(new_data)
print("Players are either 0 (not drafted) or 1 (drafted)")
print(f"The player was classified as: {predictions}")

NameError: ignored